# Technical indicator classes
In order to use lots of indicators in one strategy, we have to make a class that handles calculations for each technical indicator. That will make the code simpler.

In [5]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#plt.style.use("seaborn")
#import sys

In [6]:
#load raw data 
raw = None
def load_data():
    global raw
    start = "2022-01-31"
    end = "2023-06-30"
    raw = pd.read_csv("5m.csv", parse_dates = ["Date"], index_col = "Date").dropna()
    raw = raw.loc[start:end]
    raw = raw[["Close"]].rename(columns={"Close": "price"}) #just use these columns
    raw["returns"] = np.log(raw.price / raw.price.shift(1))
    raw

In [11]:
#Simple Moving Average
load_data() #resets data
class SMA():
    
    def __init__(self, data, SMA_S, SMA_L, column):
        self.data = data # Dataframe
        self.SMA_S = SMA_S # short SMA
        self.SMA_L = SMA_L # long SMA
        self.column = column # column to use SMA
        self.data["SMA_S"] = np.nan
        self.data["SMA_L"] = np.nan
        
    def calculate(self): #calculate for all dataframe
        self.data["SMA_S"] = self.data[self.column].rolling(self.SMA_S).mean()
        self.data["SMA_L"] = self.data[self.column].rolling(self.SMA_L).mean()
        #DONT DROP NA BECAUSE OTHER INDICATORS NEED THAT ROWS!!!
    
    def calculate_for_last_row(self): #calculate just for last row
        self.data["SMA_S"].iloc[-self.SMA_S:] = self.data[self.column].iloc[-self.SMA_S:].rolling(self.SMA_S).mean()
        self.data["SMA_L"].iloc[-self.SMA_L:] = self.data[self.column].iloc[-self.SMA_L:].rolling(self.SMA_L).mean()
    
    def strategy1(self, row):
        '''Returns predicted position (1,0 or -1)'''
        if self.data["SMA_S"].iloc[row] > self.data["SMA_L"].iloc[row]: # signal to go long
            return 1
        elif self.data["SMA_S"].iloc[row] < self.data["SMA_L"].iloc[row]: # signal to go short
            return -1
        else:
            return 0
sma = SMA(
        data = raw,
        SMA_S = 50,
        SMA_L = 200,
        column = "price"
         )
#sma.calculate() #calculate for all dataframe
sma.calculate_for_last_row() #calculate just for last row
print(sma.strategy1(-1)) #print strategy for last row
raw #notice that original dataframe is changed

1


,price,returns,SMA_S,SMA_L
Date,,,,
2022-07-12 11:15:00,19729.8,NaN,NaN,NaN
2022-07-12 11:20:00,19713.8,-0.000811,NaN,NaN
2022-07-12 11:25:00,19772.7,0.002983,NaN,NaN
2022-07-12 11:30:00,19823.3,0.002556,NaN,NaN
2022-07-12 11:35:00,19766.9,-0.002849,NaN,NaN
...,...,...,...,...
2022-08-16 04:10:00,24550.0,0.002039,NaN,NaN
2022-08-16 04:15:00,24450.0,-0.004082,NaN,NaN
2022-08-16 04:20:00,24508.3,0.002382,NaN,NaN


In [13]:
#Exponentially Weighted Moving Average
load_data() #resets data
class EWMA():
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html
    #approx average periods n are calculated by: n is approx 1/(1 - alpha)
    # => we are going to calculate alpha given n approx average periods as: alpha = 1- 1/n
    #Important: approx_avg_period are float in (1, inf). In (1,2) considers high weights for current day
    def __init__(self, data, approx_avg_period_s, approx_avg_period_l, column):
        self.data = data # Dataframe
        self.approx_avg_period_s = approx_avg_period_s
        self.approx_avg_period_l = approx_avg_period_l
        self.alpha_s = 1-1/approx_avg_period_s #alpha for short EWMA
        self.alpha_l = 1-1/approx_avg_period_l #alpha for long EWMA
        self.column = column # column to use SMA
        self.data["EWMA_S"] = np.nan
        self.data["EWMA_L"] = np.nan
        
    def calculate(self): #calculate for all dataframe
        self.data["EWMA_S"] = self.data[self.column].ewm(alpha = self.alpha_s).mean()
        self.data["EWMA_L"] = self.data[self.column].ewm(alpha = self.alpha_l).mean()
        #DONT DROP NA BECAUSE OTHER INDICATORS NEED THAT ROWS!!!
    def calculate_for_last_row(self): #calculate just for last row
        s = round(self.approx_avg_period_s)
        l = round(self.approx_avg_period_l)
        # precision. EWMA with more info gives more approx results as "calculate". recommend p = 2
        # for small s, needs more precision.
        p_s = max([100, s*2]) # min 100 of precision
        p_l = max([100, l*2]) # min 100 of precision
        #calculate EWMA and just update last row
        self.data["EWMA_S"].iloc[-1:] = self.data[self.column].iloc[-p_s:].ewm(alpha = self.alpha_s).mean()[-1]
        self.data["EWMA_L"].iloc[-1:] = self.data[self.column].iloc[-p_l:].ewm(alpha = self.alpha_l).mean()[-1]
    def strategy1(self, row):
        '''Returns predicted position (1,0 or -1)'''
        if self.data["EWMA_S"].iloc[row] > self.data["EWMA_L"].iloc[row]: # signal to go long
            return 1
        elif self.data["EWMA_S"].iloc[row] < self.data["EWMA_L"].iloc[row]: # signal to go short
            return -1
        else:
            return 0
        
ewma = EWMA(
        data = raw,
        approx_avg_period_s = 7,
        approx_avg_period_l = 99,
        column = "price"
         )
ewma.calculate() #calculate for all dataframe
ewma.calculate_for_last_row() #calculate just for last row
print(ewma.strategy1(-1)) #print strategy for last row
raw #notice that original dataframe is changed

1


,price,returns,EWMA_S,EWMA_L
Date,,,,
2022-07-12 11:15:00,19729.8,NaN,19729.800000,19729.800000
2022-07-12 11:20:00,19713.8,-0.000811,19715.800000,19713.960000
2022-07-12 11:25:00,19772.7,0.002983,19764.714035,19772.106727
2022-07-12 11:30:00,19823.3,0.002556,19814.951500,19822.782897
2022-07-12 11:35:00,19766.9,-0.002849,19773.762049,19767.464474
...,...,...,...,...
2022-08-16 04:10:00,24550.0,0.002039,24542.254466,24549.491742
2022-08-16 04:15:00,24450.0,-0.004082,24463.179209,24451.004967
2022-08-16 04:20:00,24508.3,0.002382,24501.854173,24507.721262


In [18]:
#Exponentially Weighted Moving Average
load_data() #resets data
class BollingerBands():
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html
    #approx average periods n are calculated by: n is approx 1/(1 - alpha)
    # => we are going to calculate alpha given n approx average periods as: alpha = 1- 1/n
    #Important: approx_avg_period are float in (1, inf). In (1,2) considers high weights for current day
    def __init__(self, data, column = "price", dev = 1, SMA = 50):
        self.data = data # Dataframe
        self.column = column #column used to calculate BBs
        self.dev = dev #standard deviations for BBs
        self.SMA = SMA #SMA FOR BBs
        self.last_position = 0 #saves last position
        
    def calculate(self): #calculate for all dataframe
        SM = self.data[self.column].rolling(self.SMA) #SMA one step before calculating mean()
        self.data["SMA_BBs"] = SM.mean()
        self.data["Lower"] = self.data["SMA_BBs"] - SM.std() * self.dev
        self.data["Upper"] = self.data["SMA_BBs"] + SM.std() * self.dev
        self.data["distance_SMA_BBs"] = self.data[self.column] - self.data.SMA_BBs 
        #DONT DROP NA BECAUSE OTHER INDICATORS NEED THAT ROWS!!!
    def calculate_for_last_row(self): #calculate just for last row
        SM = self.data[self.column].iloc[-self.SMA:].rolling(self.SMA)
        self.data["SMA_BBs"].iloc[-1:] = SM.mean()[-1]
        self.data["Lower"].iloc[-1:] = self.data["SMA_BBs"].iloc[-1:] - SM.std()[-1] * self.dev
        self.data["Upper"].iloc[-1:] = self.data["SMA_BBs"].iloc[-1:] + SM.std()[-1] * self.dev
        
    def strategy1(self, row):
        '''Returns predicted position (1,0 or -1)'''
        ### How to evaluate vectorized strategy ###
        #self.data["position"] = np.where(self.data[self.column] < self.data.Lower, 1, np.nan)
        #self.data["position"] = np.where(self.data[self.column] > self.data.Upper, -1, self.data["position"])
        #self.data["position"] = np.where(self.data.distance * self.data.distance.shift(1) < 0, 0, self.data["position"])
        #self.data["position"] = self.data.position.ffill().fillna(0) 
                
        if self.data[self.column].iloc[row] < self.data.Lower.iloc[row]:
            self.last_position = 1
            return self.last_position
        elif self.data[self.column].iloc[row] > self.data.Upper.iloc[row]:
            self.last_position = -1 
            return self.last_position
        elif row != 0 and self.data["distance_SMA_BBs"].iloc[row] * self.data["distance_SMA_BBs"].iloc[row-1] < 0:
            self.last_position = 0
            return self.last_position
        else:
            return self.last_position
        
        
bbs = BollingerBands(
        data = raw,
        dev = 1, 
        SMA = 50,
        column = "price"
         )
bbs.calculate() #calculate for all dataframe
bbs.calculate_for_last_row() #calculate just for last row
print(bbs.strategy1(-1)) #print strategy for last row
raw #notice that original dataframe is changed

0


,price,returns,SMA_BBs,Lower,Upper,distance_SMA_BBs
Date,,,,,,
2022-07-12 11:15:00,19729.8,NaN,NaN,NaN,NaN,NaN
2022-07-12 11:20:00,19713.8,-0.000811,NaN,NaN,NaN,NaN
2022-07-12 11:25:00,19772.7,0.002983,NaN,NaN,NaN,NaN
2022-07-12 11:30:00,19823.3,0.002556,NaN,NaN,NaN,NaN
2022-07-12 11:35:00,19766.9,-0.002849,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2022-08-16 04:10:00,24550.0,0.002039,24324.026,24141.569419,24506.482581,225.974
2022-08-16 04:15:00,24450.0,-0.004082,24327.712,24144.597338,24510.826662,122.288
2022-08-16 04:20:00,24508.3,0.002382,24335.530,24153.234947,24517.825053,172.770


In [13]:
BBS.SM

Rolling [window=50,center=False,axis=0,method=single]